# Transformer

<div style="display: flex; align-items: center;">
    <img src="../imgs/Transformer.jpg" alt="Your Image" width="300" style="margin-right: 20px;">
    <div>
        <p>The Transformer model is a revolutionary deep learning architecture that, with its unique self attention mechanism as its core, completely changes the way sequence modeling is done. This mechanism allows the model to consider all elements in parallel when processing sequences, rather than gradually processing them in order like traditional recurrent neural networks, greatly improving computational efficiency. Through multi head attention, Transformer can simultaneously capture sequence information from different perspectives, enhancing the model's ability to learn complex features.</p>
        <p>In addition to self attention mechanism, Transformer also introduces positional encoding to solve the problem of element order in sequences, which is crucial for maintaining the temporal sensitivity of sequence data. In each encoder and decoder layer of the model, the output of the self attention layer is transmitted to the feedforward network for further feature extraction and processing. In order to improve the training stability of deep networks, Transformer adopts layer normalization technology and alleviates the problem of gradient vanishing through residual connections, making the training of deep networks more feasible.</p>
        <p>These design features of the Transformer model have quickly made it mainstream in the field of natural language processing, especially in tasks such as machine translation, text summarization, and question answering systems. Its flexibility and powerful representation ability have also shown wide application potential in other fields such as speech recognition and image processing, making it one of the most influential models in the current field of deep learning.</p>
    </div>
</div>

# Vision Transformer

<div style="display: flex; align-items: center;">
    <img src="../imgs/ViT.jpg" alt="Your Image" width="600" style="margin-right: 20px;">
    <div>
        <p>ViT (vision transformer) is a model proposed by Google in 2020 that directly applies transformer to image classification. Many subsequent works have been improved based on ViT. The idea of ViT is simple: directly divide the image into fixed size patches, and then obtain patch embeddings through linear transformation, which is similar to NLP's words and word embeddings. Since the input of the transformer is a sequence of token embeddings, the patch embeddings of the image can be fed into the transformer for feature extraction and classification. As shown in the schematic diagram of the ViT model, in fact, the ViT model only uses the Encoder of the transformer to extract features (the original transformer also has a decoder section, which is used to implement sequence to sequence, such as machine translation).</p>
        <p></p>
    </div>
</div>

Before building the model, let's review the parameters of the new layer:

**nn.TransformerEncoderLayer** is a module implemented in PyTorch that standardizes the common functions for a Transformer encoder layer. This layer is a building block for the Transformer encoder architecture, which can be stacked multiple times to form the full Transformer encoder. The common parameters for nn.TransformerEncoderLayer are as follows:

`d_model`: The feature dimension of the input and output of the layer, which corresponds to the number of expected features in the input (seq_len, batch, d_model).

`nhead`: The number of heads for the multiheadattention mechanisms in the layer.

`dim_feedforward`: The dimension of the feedforward network inside the transformer block.

`dropout`: (optional) The dropout probability for the dropout layer inside the transformer block. Default is 0.1.

`activation`: (optional) The activation function to use in the feedforward network. Default is "relu".

`batch_first`: (optional) If True, the input and output tensors are provided as (batch, seq, feature). If False, they are provided as (seq, batch, feature). Default is False.

`bias`: (optional) Whether to include bias terms in the attention scores and feedforward network. Default is True.

`norm_eps`: (optional) The epsilon constant to use for the layer normalization. Default is 1e-5.

`norm_first`: (optional) Whether to perform layer normalization before the feedforward network. If False, the feedforward network is performed first, followed by layer normalization. Default is False.

## Train ViT on CIFAR100

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ViT(nn.Module):
    def __init__(self, input_channels, image_size, patch_size, num_classes, num_heads, num_encoder_layers, dim_feedforward):
        super(ViT, self).__init__()
        self.image_size = image_size
        self.patch_size = patch_size
        self.num_classes = num_classes
        self.num_patches = (image_size // patch_size) ** 2
        self.patch_dim = input_channels * patch_size * patch_size

        self.conv1 = nn.Conv2d(input_channels, 768, kernel_size=patch_size, stride=patch_size, bias=False)  # Patch embedding
        self.positional_encoding = nn.Parameter(self._generate_positional_encoding(self.num_patches, 768))

        encoder_layer = nn.TransformerEncoderLayer(d_model=768, nhead=num_heads, dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers)

        self.fc = nn.Linear(768, num_classes)
        
    def _generate_positional_encoding(self, num_patches, d_model, dtype=torch.float):
        """
        Generate a 2D positional encoding as per the Transformer model
        """
        position = torch.arange(num_patches, dtype=dtype).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=dtype) * -(torch.log(torch.tensor(10000.0)) / d_model))
        positional_encoding = torch.zeros((num_patches, d_model), dtype=dtype)
        positional_encoding[:, 0::2] = torch.sin(position * div_term)
        positional_encoding[:, 1::2] = torch.cos(position * div_term)
        return positional_encoding.unsqueeze(0)

    def forward(self, x):
        x = self.conv1(x).flatten(2)  # Patchify the image
        x = x.transpose(1, 2)  # Change to N C L
        x += self.positional_encoding[:, :x.size(1), :].to(x.device)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)  # Global average pooling
        x = self.fc(x)
        return x

In [2]:
import sys
import torch.nn as nn
sys.path.append('../tools')
from CIFAR10 import CIFAR10Trainer
from CIFAR100 import CIFAR100Trainer

Vision Transformers (ViTs), while highly effective on large-scale datasets, may underperform compared to Convolutional Neural Networks (CNNs) on smaller or simpler datasets without pre-training due to several factors. ViTs require substantial data to leverage their large model capacity, which can lead to overfitting on limited data. Their design focuses on capturing global dependencies, which might be excessive for the local pattern recognition needed in smaller datasets. Additionally, without the feature-rich initialization provided by pre-training, ViTs struggle to learn from scratch, unlike CNNs that are inherently efficient and can quickly adapt to available data due to their architectural advantages in processing spatial hierarchies. So in this chapter, we will only introduce the concept and structure of ViT. For application scenarios, you can explore them yourself, and I will also try to demonstrate them in later chapters.

In [ ]:
model = ViT(input_channels=3, image_size=32, patch_size=8, num_classes=100, num_heads=8, num_encoder_layers=3, dim_feedforward=2048)
trainer = CIFAR100Trainer(model, loss='CE', lr=0.01, optimizer='SGD', batch_size=128, epoch=50, model_type='classification')
trainer.train()
trainer.test()

Files already downloaded and verified
Files already downloaded and verified
2024-05-20 18:47:33
Epoch 1 / 50


[Valid]: 100%|███████████████████████| 40/40 [00:02<00:00, 16.99it/s, val_acc=9.78, val_loss=0.0313]


2024-05-20 18:48:01
Epoch 2 / 50


[Valid]: 100%|███████████████████████| 40/40 [00:02<00:00, 13.61it/s, val_acc=12.5, val_loss=0.0298]


2024-05-20 18:48:31
Epoch 3 / 50


...

[Valid]: 100%|███████████████████████| 40/40 [00:03<00:00, 12.62it/s, val_acc=41.1, val_loss=0.0223]


2024-05-20 19:13:29
Epoch 49 / 50


[Valid]: 100%|███████████████████████| 40/40 [00:03<00:00, 12.77it/s, val_acc=41.9, val_loss=0.0226]


2024-05-20 19:14:03
Epoch 50 / 50


[Valid]: 100%|███████████████████████| 40/40 [00:03<00:00, 11.02it/s, val_acc=40.6, val_loss=0.0232]
